# 02 Chat Completion | 01 REST Call

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Intro

Chat models take a series of messages as input, and return a model-generated message as output. The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).

## Step 1:  Setup Parameters

In [1]:
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Text.Json, 7.0.3"
#r "nuget: Newtonsoft.Json, 13.0.1"
using DotNetEnv;

using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;

static string _configurationFile = @"../Configuration/application.env";
Env.Load(_configurationFile);

string apiBase = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT"); 
string apiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY"); 
string deploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME"); 
string apiVersion = "2023-07-01-preview";


Installed Packages DotNetEnv, 2.5.0 Newtonsoft.Json, 13.0.1 System.Text.Json, 7.0.3

## Step 2:  Create completions for chat messages with GPT models

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance.

System prompt: `"You are an AI assistance who helps finding information around international sport events"`

Prompt: `"Who won the super bowl 2024?"`

In [2]:

var requestPayload = new JsonObject
{
    { "messages", new JsonArray
        {
            new JsonObject
            {
                { "role", "system" },
                { "content", "You are an AI assistance who helps finding information around international sport events" }
                
            },
            new JsonObject
            {
                { "role", "user" },
                { "content", "Who won the super bowl 2024?" }
            }
        }
    },
    { "max_tokens", 200 },
    { "temperature", 0.7 },
    { "frequency_penalty", 0 },
    { "presence_penalty", 0 },
    { "top_p", 0.95 },
    { "stop", null }
};

string payload = JsonSerializer.Serialize(requestPayload, new JsonSerializerOptions
{
    WriteIndented = true // Optional: to make the JSON string more readable
});

Console.WriteLine($"Request payload defined ...");

Request payload defined ...


## Step 3:  Call Azure OpenAI REST endpoint

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance.

In [3]:
string endpoint = $"{apiBase}openai/deployments/{deploymentName}/chat/completions?api-version={apiVersion}";

using (HttpClient httpClient = new HttpClient())
{
    httpClient.BaseAddress = new Uri(endpoint);
    httpClient.DefaultRequestHeaders.Add("api-key",apiKey);
    httpClient.DefaultRequestHeaders.Accept.Add(new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

    var stringContent = new StringContent(payload, Encoding.UTF8, "application/json");

    var response = await httpClient.PostAsync(endpoint, stringContent);

    if (response.IsSuccessStatusCode)
    {
        using (var responseStream = await response.Content.ReadAsStreamAsync())
        {
            // Parse the JSON response using JsonDocument
            using (var jsonDoc = await JsonDocument.ParseAsync(responseStream))
            {
                // Access the message content dynamically
                var root = jsonDoc.RootElement;
                var messageContent = root.GetProperty("choices")[0].GetProperty("message").GetProperty("content").GetString();

                // Output the message content
                Console.WriteLine("Output: " + messageContent);
            }
        }
    }
    else
    {
        Console.WriteLine($"Error: {response}");
    }
}

Error: StatusCode: 400, ReasonPhrase: 'model_error', Version: 1.1, Content: System.Net.Http.HttpConnectionResponseContent, Headers:
{
  request-id: 9f02e439-8cab-48b7-9ab9-45783be142de
  X-Content-Type-Options: nosniff
  apim-request-id: 9f02e439-8cab-48b7-9ab9-45783be142de
  x-ms-region: Sweden Central
  x-ms-rai-invoked: true
  X-Request-ID: 090df953-bdc2-44f0-a2c3-8f90274940c8
  ms-azureml-model-error-reason: model_error
  ms-azureml-model-error-statuscode: 400
  x-ms-client-request-id: 9f02e439-8cab-48b7-9ab9-45783be142de
  Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
  azureml-model-session: mmapi-v0202-0508-c
  api-supported-versions: 1
  x-envoy-upstream-service-time: 36
  x-ratelimit-remaining-requests: 0
  x-ratelimit-remaining-tokens: 800
  Date: Tue, 18 Jun 2024 11:30:45 GMT
  Content-Length: 191
  Content-Type: application/json; charset=utf-8
}
